In [ ]:
# !git clone https://github.com/poloclub/llm-tutorial-data.git

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Fine tuning llama 2 model/")

In [ ]:
os.getcwd()

'/content/drive/MyDrive/Fine tuning llama 2 model'

In [ ]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install BitsandBytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
!pip install GPUtil

In [ ]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [ ]:
import torch
import os
from huggingface_hub import notebook_login
import transformers
from transformers import AutoModelForCausalLM, LlamaTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager() #to show output beautifully

In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `login2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `login2`


# load the model using BitsandBytesConfig (4bit) [BitsAndBytes](https://huggingface.co/docs/transformers/v4.51.3/en/main_classes/quantization#transformers.BitsAndBytesConfig)

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"


bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,    #load model in 4bit quant
    bnb_4bit_use_double_quant=True, #double 4bit quant on loaded model
    bnb_4bit_quant_type="nf4",      # normal float 4 operation
    bnb_4bit_compute_dtype=torch.bfloat16
)

#load model
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# Load dataset

In [ ]:
!git clone https://github.com/poloclub/llm-tutorial-data.git

Cloning into 'llm-tutorial-data'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 0), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 61.41 KiB | 849.00 KiB/s, done.


In [ ]:
from datasets import load_dataset

In [ ]:
training_data = load_dataset('text', data_files= {'train': ["./llm-tutorial-data/hawaii_wf_4.txt",
                                                           "./llm-tutorial-data/hawaii_wf_8.txt"]},
                             split='train')

training_data

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 346
})

In [ ]:
training_data['text'][0:2]

['In the early morning of August 9, 2023, the officer further assisted in coordinating transportation for people who',
 'had taken refuge in the ocean to escape the fire, ensuring they reached the emergency shelter safely.']

In [ ]:
tokenizer= LlamaTokenizer.from_pretrained(base_model_id,
                                          add_eos_token=True,
                                          add_bos_token=True,
                                          trust_remote_code=True)

print("All special tokens : ",  tokenizer.all_special_tokens)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token' : tokenizer.eos_token})


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

All special tokens :  ['<s>', '</s>', '<unk>']


In [ ]:
tokenizer #now we have added the padding token as eos_token

LlamaTokenizer(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
#tokenize the traininig_Dataset

tokenizer_trained_dataset= []

for row in training_data['text']:
    tokenizer_trained_dataset.append(tokenizer(row))

tokenizer_trained_dataset[:2]

[{'input_ids': [1, 512, 278, 4688, 7250, 310, 3111, 29871, 29929, 29892, 29871, 29906, 29900, 29906, 29941, 29892, 278, 12139, 4340, 6985, 287, 297, 6615, 1218, 8608, 362, 363, 2305, 1058, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [1, 750, 4586, 25447, 297, 278, 23474, 304, 10169, 278, 3974, 29892, 5662, 3864, 896, 7450, 278, 11176, 14703, 27709, 23511, 29889, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}]

In [ ]:
# training_data.map(lambda x: tokenizer(x['text']), batched=True) #another way

## chat prompt template example

In [ ]:
messages= [{'role':'system', 'content': "You are an  helpfule assistant"},
           {'role': 'user', 'content': 'tell me the story'},
           {'role': 'assistant', 'content': "Nanna munna rai hu"}]

print(tokenizer.apply_chat_template(messages, tokenize=False))

<s>[INST] <<SYS>>
You are an  helpfule assistant
<</SYS>>

tell me the story [/INST] Nanna munna rai hu </s>


In [ ]:
user_question= "What was the initial assignment of the Lahaina Patrol supervisor on August 8, 2023?"
eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")


with torch.no_grad():
  print(tokenizer.decode(model.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: What was the initial assignment of the Lahaina Patrol supervisor on August 8, 2023? Just answer this question accurately and concisely.
sure! Here is the information you requested:

The initial assignment of the Lahaina Patrol supervisor on August 8, 2023, was to patrol the area around Lahaina Harbor and respond to any incidents or calls for service in the area.


# Model Training

In [ ]:
model  #("q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096

In [ ]:
model.gradient_checkpointing_enable() #to reduce memory IT's already available in preparekk bit training function
model = prepare_model_for_kbit_training(model)

lora_config= LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config) #lora model

model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

# Training

In [ ]:
len(tokenizer_trained_dataset)/2

173.0

In [ ]:
trainer= transformers.Trainer(model=model,
                     train_dataset= tokenizer_trained_dataset,
                     args= transformers.TrainingArguments(
                                        output_dir="./finetuned_llama",
                                        per_device_train_batch_size=4,
                                        gradient_accumulation_steps=2,
                                        learning_rate=1e-4,
                                        num_train_epochs=4,
                                        logging_steps=10,
                                        save_strategy="epoch",
                                        push_to_hub=False,
                                        logging_dir="./log",
                                        logging_strategy= "steps",
                                        bf16=False,
                                        optim='paged_adamw_8bit',
                                        save_steps=50,
                                        max_steps=200,
                                        run_name= "second-hawaii-fire-llama-training"),
                     data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shubhamchau (shubhamchau-hsm-s-ssgbcoet-bhusawal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.744500
20,3.180900
30,2.863500
40,2.863300
50,2.513800
60,2.405400
70,2.278500
80,2.243600
90,2.107000
100,1.872100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=200, training_loss=2.036556100845337, metrics={'train_runtime': 1462.0547, 'train_samples_per_second': 1.094, 'train_steps_per_second': 0.137, 'total_flos': 1931561494806528.0, 'train_loss': 2.036556100845337, 'epoch': 4.551724137931035})

# Combine the adapter with the base model to make the complete model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"


bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,    #load model in 4bit quant
    bnb_4bit_use_double_quant=True, #double 4bit quant on loaded model
    bnb_4bit_quant_type="nf4",      # normal float 4 operation
    bnb_4bit_compute_dtype=torch.bfloat16
)

#load model
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer= LlamaTokenizer.from_pretrained(base_model_id,
                                          add_eos_token=True,
                                          add_bos_token=True,
                                          trust_remote_code=True)

print("All special tokens : ",  tokenizer.all_special_tokens)

All special tokens :  ['<s>', '</s>', '<unk>']


In [ ]:
modelFinetuned= PeftModel.from_pretrained(model, "./finetuned_llama/checkpoint-132")
# modelFinetuned

In [ ]:
user_question = "When did Hawaii wildfires start?"

eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: When did Hawaii wildfires start? Just answer this question accurately and concisely.
ily of the Maui Fire Department. The Maui Fire Department is a department within the Government of Maui. The Maui Fire Department is responsible for responding to fire-related incidents in Maui County, Hawaii. The Maui Fire Department is headquartered at the Maui Fire Department headquarters located at 10 Kuialua Street, Kahului, Maui, Hawaii 96732. The Maui Fire Department operates 24 hours a day, 7 days a week. The Maui Fire Department is led by the Fire Chief, who is appointed by the Mayor of Maui. The Maui Fire Department consists of 23 fire stations located throughout Maui County. The Maui Fire Department is staffed by firefighters who are trained to respond to fires, medical emergencies, motor vehicle accidents, and other emergencies. The Maui Fire Department also has a fire prevention bureau, which is responsible for conducting fire safety inspections and investigating the cause of fir

In [ ]:
modelFinetuned.save_pretrained("./llama2-finetuned_full-model")

In [ ]:
from huggingface_hub import upload_folder

In [ ]:
# upload_folder(
#     folder_path="./llama2-finetuned_full-model",  # Path to your model folder
#     repo_id="Shubhu07/llama2-7B-chat-lora-hawaiifire",    # Replace with your Hugging Face username and desired model name
#     token="hf_qeCUvsIzVeBK.............",             # Your Hugging Face token
#     commit_message="Initial model upload"
# )